In [1]:
import os
import sys

sys.path.insert(0, "/usr/local/spark/python")
sys.path.insert(0, '/usr/local/spark/python/lib/py4j-0.9-src.zip')
from pyspark import SparkContext
from pyspark import SparkConf

conf = SparkConf().setMaster("spark://spark-master.docpyml:7077").setAppName('pyspark')
sc = SparkContext(conf=(conf))

In [6]:
!/usr/bin/unzip mllib.zip 

Archive:  mllib.zip
   creating: mllib/
   creating: mllib/data/
   creating: mllib/data/als/
  inflating: mllib/data/als/test.data  
   creating: mllib/data/lr-data/
  inflating: mllib/data/lr-data/random.data  
   creating: mllib/data/ridge-data/
  inflating: mllib/data/ridge-data/lpsa.data  
  inflating: mllib/data/sample_naive_bayes_data.txt  
  inflating: mllib/data/sample_svm_data.txt  


In [7]:
!mv mllib /shared/

In [16]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint

# Load and parse the data
def parsePoint(line):
    values = [float(x) for x in line.split(' ')]
    return LabeledPoint(values[0], values[1:])

data = sc.textFile("file:///shared/mllib/data/sample_svm_data.txt")
parsedData = data.map(parsePoint)

# Build the model
model = SVMWithSGD.train(parsedData, iterations=100)

# Evaluating the model on training data
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda t: t[0] != t[1]).count() / float(parsedData.count())
print("Training Error = " + str(trainErr))

Training Error = 0.38198757763975155
